<a href="https://colab.research.google.com/github/HssahSaad/Data_Science_AI/blob/main/Text_Summarization_2_Hssah_Alsherihi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
%pip install datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("CUTD/news_articles_df")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'summarizer', 'text'],
        num_rows: 8378
    })
})

In [4]:
ds = ds["train"]

In [5]:
# Drop unecessary columns
ds = ds.remove_columns("Unnamed: 0")
ds

Dataset({
    features: ['summarizer', 'text'],
    num_rows: 8378
})

In [6]:
# Split the dataset with 20% for test
ds.train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['summarizer', 'text'],
        num_rows: 6702
    })
    test: Dataset({
        features: ['summarizer', 'text'],
        num_rows: 1676
    })
})

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [7]:
from transformers import AutoTokenizer

checkpoint ="google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [8]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summarizer"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized = ds.map(preprocess_function, batched=True)

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.310400,0.295443
2,0.302900,0.287510
3,0.293700,0.287538
4,0.293700,0.286321


TrainOutput(global_step=4192, training_loss=0.3065106805044276, metrics={'train_runtime': 1550.216, 'train_samples_per_second': 21.618, 'train_steps_per_second': 2.704, 'total_flos': 3288382832640000.0, 'train_loss': 0.3065106805044276, 'epoch': 4.0})

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [25]:
text = "وتحفل معلقة الأعشى بفيض من الصور والتشابيه والأوصاف الدقيقة اللينة لمفاتن المرأة وحوار الحب ولقاء المتعة وتصوير علاقات العشق الفاشلة، والحديث عن مجلس الشّراب مع الصحاب والقيان، أما موضوع الهجاء فهو موجه إلى يزيد بن شيبان، ولوم الشاعر له وافتخاره على قومه بالبلاء الأوفى بالحرب، والانتهاء إلى الوعيد والتهديد بالثأر. يحاول الأعشى في القصيدة أن يبين الحبّ الفاشل، وكيف أن المرء قد يعلق بفتاة لا تحبه بل تحب سواه، والآخر المحبوب قد لا يحبّها، وكيف تتشابك مثل هذه العلاقات الفاشلة."

In [35]:
from transformers import pipeline

summarizer = pipeline("summarization", model='google-t5/t5-small' , tokenizer='google-t5/t5-small', max_length=83)
summarizer(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': ' ,  and .  () ()( )( . .'}]